In [2]:
import pandas as pd
from datetime import datetime
import networkx as nx
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [3]:
def read_data(file_path):
    try:
        data = pd.read_excel(file_path, engine='openpyxl')

        if 'Дата_Рождения' in data.columns:
            data['Дата_Рождения'] = pd.to_datetime(data['Дата_Рождения'], errors='coerce')

            today = datetime.today()
            data['Возраст'] = data['Дата_Рождения'].apply(
                lambda x: (today - x).days // 365 if pd.notnull(x) else None
            )               

        return data
    except Exception as e:
        print(f"Ошибка при чтении файла: {e}")
        return None

In [4]:
fenotype_data = read_data('./../data/Датасет на хакатон.xlsx')
fenotype_data.fillna(0, inplace=True)
fenotype_data.head(7)

,ID_особи,Пол,Порода,Дата_Рождения,Родитель_папа,Родитель_мама,Удой л/день,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,Здоровье (1-10),Фертильность (%),Генетическая ценность (баллы),Возраст
0,1,Самка,Симентальская,2022-12-22,1996,150,0.000000,4,0.17,0.00,8,76.0,93,1
1,2,Самка,Герефорд,2018-02-07,7154,6500,37.478242,4,0.15,1.18,5,0.0,87,6
2,3,Самец,Швицкая,2023-07-04,9107,1784,21.730202,3,0.07,0.00,5,0.0,70,1
3,4,Самец,Герефорд,2022-10-08,3959,1785,22.625400,2,0.12,0.00,9,82.0,84,2
4,5,Самка,Айрширская,2020-10-07,6921,4409,0.000000,3,0.14,0.00,6,0.0,87,4
5,6,Самка,Голштинская,2019-08-26,7942,7035,35.590611,5,0.01,0.00,7,88.0,86,5
6,7,Самец,Герефорд,2022-03-04,9472,9004,0.000000,4,0.11,0.00,5,0.0,85,2


In [5]:
genotype_data = read_data('./../data/Генетические мутации хакатон.xlsx')
genotype_data.head(7)

,mutation_id,chrom,pos,ref,alt,Признак,beta,Генотип коровы,ID_особи
0,mut_000001,4,39128359,G,T,Фертильность (%),9.52,T/T,1
1,mut_000002,6,5624123,T,A,Прирост веса кг/день,-6.76,A/A,1
2,mut_000003,26,41747388,T,A,Здоровье (1-10),-2.64,T/T,1
3,mut_000004,27,25531753,C,G,Коэффициент инбридинга (F),1.04,C/C,2
4,mut_000005,105,9093860,T,C,Упитанность,0.15,T/C,2
5,mut_000006,103,23951303,T,G,Удой л/день,-9.32,T/T,2
6,mut_000007,41,20770253,A,C,Генетическая ценность (баллы),-8.65,A/C,2


In [6]:
merged_data = pd.merge(genotype_data, fenotype_data, on='ID_особи')
merged_data.fillna(0, inplace=True)
merged_data.head(7)

,mutation_id,chrom,pos,ref,alt,Признак,beta,Генотип коровы,ID_особи,Пол,...,Родитель_папа,Родитель_мама,Удой л/день,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,Здоровье (1-10),Фертильность (%),Генетическая ценность (баллы),Возраст
0,mut_000001,4,39128359,G,T,Фертильность (%),9.52,T/T,1,Самка,...,1996,150,0.000000,4,0.17,0.00,8,76.0,93,1
1,mut_000002,6,5624123,T,A,Прирост веса кг/день,-6.76,A/A,1,Самка,...,1996,150,0.000000,4,0.17,0.00,8,76.0,93,1
2,mut_000003,26,41747388,T,A,Здоровье (1-10),-2.64,T/T,1,Самка,...,1996,150,0.000000,4,0.17,0.00,8,76.0,93,1
3,mut_000004,27,25531753,C,G,Коэффициент инбридинга (F),1.04,C/C,2,Самка,...,7154,6500,37.478242,4,0.15,1.18,5,0.0,87,6
4,mut_000005,105,9093860,T,C,Упитанность,0.15,T/C,2,Самка,...,7154,6500,37.478242,4,0.15,1.18,5,0.0,87,6
5,mut_000006,103,23951303,T,G,Удой л/день,-9.32,T/T,2,Самка,...,7154,6500,37.478242,4,0.15,1.18,5,0.0,87,6
6,mut_000007,41,20770253,A,C,Генетическая ценность (баллы),-8.65,A/C,2,Самка,...,7154,6500,37.478242,4,0.15,1.18,5,0.0,87,6


In [7]:
def check_mutation(genotype, ref, alt):
    if genotype == f"{alt}/{alt}":
        return 1
    elif genotype == f"{ref}/{ref}":
        return 0
    elif genotype == f"{ref}/{alt}" or genotype == f"{alt}/{ref}":
        return 0.5
    else:
        return -1
    
merged_data['is_mutation'] = merged_data.apply(lambda row: check_mutation(row['Генотип коровы'], row['ref'], row['alt']), axis = 1)
merged_data.head(7)

,mutation_id,chrom,pos,ref,alt,Признак,beta,Генотип коровы,ID_особи,Пол,...,Родитель_мама,Удой л/день,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,Здоровье (1-10),Фертильность (%),Генетическая ценность (баллы),Возраст,is_mutation
0,mut_000001,4,39128359,G,T,Фертильность (%),9.52,T/T,1,Самка,...,150,0.000000,4,0.17,0.00,8,76.0,93,1,1.0
1,mut_000002,6,5624123,T,A,Прирост веса кг/день,-6.76,A/A,1,Самка,...,150,0.000000,4,0.17,0.00,8,76.0,93,1,1.0
2,mut_000003,26,41747388,T,A,Здоровье (1-10),-2.64,T/T,1,Самка,...,150,0.000000,4,0.17,0.00,8,76.0,93,1,0.0
3,mut_000004,27,25531753,C,G,Коэффициент инбридинга (F),1.04,C/C,2,Самка,...,6500,37.478242,4,0.15,1.18,5,0.0,87,6,0.0
4,mut_000005,105,9093860,T,C,Упитанность,0.15,T/C,2,Самка,...,6500,37.478242,4,0.15,1.18,5,0.0,87,6,0.5
5,mut_000006,103,23951303,T,G,Удой л/день,-9.32,T/T,2,Самка,...,6500,37.478242,4,0.15,1.18,5,0.0,87,6,0.0
6,mut_000007,41,20770253,A,C,Генетическая ценность (баллы),-8.65,A/C,2,Самка,...,6500,37.478242,4,0.15,1.18,5,0.0,87,6,0.5


In [8]:
def calculate_final_trait(row):
    base_value = row[row['Признак']]
    return base_value + row['beta'] * row['is_mutation']

merged_data['Итоговое значение признака'] = merged_data.apply(calculate_final_trait, axis = 1)
merged_data.head(7)

,mutation_id,chrom,pos,ref,alt,Признак,beta,Генотип коровы,ID_особи,Пол,...,Удой л/день,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,Здоровье (1-10),Фертильность (%),Генетическая ценность (баллы),Возраст,is_mutation,Итоговое значение признака
0,mut_000001,4,39128359,G,T,Фертильность (%),9.52,T/T,1,Самка,...,0.000000,4,0.17,0.00,8,76.0,93,1,1.0,85.520000
1,mut_000002,6,5624123,T,A,Прирост веса кг/день,-6.76,A/A,1,Самка,...,0.000000,4,0.17,0.00,8,76.0,93,1,1.0,-6.760000
2,mut_000003,26,41747388,T,A,Здоровье (1-10),-2.64,T/T,1,Самка,...,0.000000,4,0.17,0.00,8,76.0,93,1,0.0,8.000000
3,mut_000004,27,25531753,C,G,Коэффициент инбридинга (F),1.04,C/C,2,Самка,...,37.478242,4,0.15,1.18,5,0.0,87,6,0.0,0.150000
4,mut_000005,105,9093860,T,C,Упитанность,0.15,T/C,2,Самка,...,37.478242,4,0.15,1.18,5,0.0,87,6,0.5,4.075000
5,mut_000006,103,23951303,T,G,Удой л/день,-9.32,T/T,2,Самка,...,37.478242,4,0.15,1.18,5,0.0,87,6,0.0,37.478242
6,mut_000007,41,20770253,A,C,Генетическая ценность (баллы),-8.65,A/C,2,Самка,...,37.478242,4,0.15,1.18,5,0.0,87,6,0.5,82.675000


In [9]:
def calculate_penalty(value, optimal_range, weight):
    if isinstance(optimal_range, tuple):
        lower, upper = optimal_range
        if value < lower:
            return (lower - value) * weight
        elif value > upper:
            return (value - upper) * weight
    elif isinstance(optimal_range, float) or isinstance(optimal_range, int):
        return abs(value - optimal_range) * weight
    return 0

In [10]:
def evaluate_cow(cow, common_weights, direction_weights, direction):
    penalty = 0
    penalty += calculate_penalty(cow['Коэффициент инбридинга (F)'], (0, 0.24), common_weights['Коэффициент инбридинга (F)']) 
    penalty += calculate_penalty(cow['Здоровье (1-10)'], (8, 10), common_weights['Здоровье (1-10)']) 
    penalty += calculate_penalty(cow['Упитанность'], (3, 4), common_weights['Упитанность']) 
    penalty += calculate_penalty(cow['Фертильность (%)'], 70, common_weights['Фертильность (%)']) 
    penalty += calculate_penalty(cow['Генетическая ценность (баллы)'], (80, 100), common_weights['Генетическая ценность (баллы)'])

    if direction in direction_weights: 
        dir_weights = direction_weights[direction] 
        penalty += calculate_penalty(cow['Удой л/день'], 30, dir_weights['Удой л/день']) # Укажите свои оптимальные значения 
        penalty += calculate_penalty(cow['Прирост веса кг/день'], 0.7, dir_weights['Прирост веса кг/день']) 
    return penalty

In [11]:
my_cow = {
    'ID_особи': 2,
    'Пол': 'Самка',	
    'Порода': 'Айрширская',
    'Дата_Рождения': '2021-10-19',	
    'Родитель_папа': 3532,
    'Родитель_мама': 6271,
    'Удой л/день': 20,
    'Упитанность': 4,
    'Коэффициент инбридинга (F)': 0.02,	
    'Прирост веса кг/день': 1.22,	
    'Здоровье (1-10)': 5,	
    'Фертильность (%)': 85,	
    'Генетическая ценность (баллы)': 70,
    'Возраст': 3,
}

common_weights = {
    "Коэффициент инбридинга (F)": 2.0,
    "Здоровье (1-10)": 1.5,
    "Упитанность": 1.2,
    "Возраст": 1.0,
    "Фертильность (%)": 1.7,
    "Генетическая ценность (баллы)": 1.4
}

direction_weights = {
    'молочное': {
        'Удой л/день': 2.0,
        'Прирост веса кг/день': 1.0
    },
    'мясное': {
        'Удой л/день': 1.0,
        'Прирост веса кг/день': 2.0
    },
    'комбинированное': {
        'Удой л/день': 1.5,
        'Прирост веса кг/день': 1.5
    },

}


direction = 'молочное'
fenotype_data['Penalty'] = fenotype_data.apply(
    lambda row: evaluate_cow(row, common_weights, direction_weights, direction),
    axis = 1
)

fenotype_data.head(7)

,ID_особи,Пол,Порода,Дата_Рождения,Родитель_папа,Родитель_мама,Удой л/день,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,Здоровье (1-10),Фертильность (%),Генетическая ценность (баллы),Возраст,Penalty
0,1,Самка,Симентальская,2022-12-22,1996,150,0.000000,4,0.17,0.00,8,76.0,93,1,70.900000
1,2,Самка,Герефорд,2018-02-07,7154,6500,37.478242,4,0.15,1.18,5,0.0,87,6,138.936484
2,3,Самец,Швицкая,2023-07-04,9107,1784,21.730202,3,0.07,0.00,5,0.0,70,1,154.739596
3,4,Самец,Герефорд,2022-10-08,3959,1785,22.625400,2,0.12,0.00,9,82.0,84,2,37.049199
4,5,Самка,Айрширская,2020-10-07,6921,4409,0.000000,3,0.14,0.00,6,0.0,87,4,182.700000
5,6,Самка,Голштинская,2019-08-26,7942,7035,35.590611,5,0.01,0.00,7,88.0,86,5,45.181223
6,7,Самец,Герефорд,2022-03-04,9472,9004,0.000000,4,0.11,0.00,5,0.0,85,2,184.200000


In [12]:
my_cow_penalty = evaluate_cow(pd.Series(my_cow), common_weights, direction_weights, direction)
my_cow_penalty

64.52

In [17]:
def calculate_compatibility(cow, my_cow, cow_penalty, my_cow_penalty):
    if cow['Пол'] == my_cow['Пол']:
        return 0
    return 100 - (abs(cow_penalty - my_cow_penalty) / max(cow_penalty, my_cow_penalty)) * 100

In [18]:
fenotype_data['Compatibility'] = fenotype_data.apply( 
    lambda row: calculate_compatibility(row, pd.Series(my_cow), row['Penalty'], my_cow_penalty), 
    axis = 1
)
fenotype_data.head(7)

,ID_особи,Пол,Порода,Дата_Рождения,Родитель_папа,Родитель_мама,Удой л/день,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,Здоровье (1-10),Фертильность (%),Генетическая ценность (баллы),Возраст,Penalty,Compatibility
0,1,Самка,Симентальская,2022-12-22,1996,150,0.000000,4,0.17,0.00,8,76.0,93,1,70.900000,0.000000
1,2,Самка,Герефорд,2018-02-07,7154,6500,37.478242,4,0.15,1.18,5,0.0,87,6,138.936484,0.000000
2,3,Самец,Швицкая,2023-07-04,9107,1784,21.730202,3,0.07,0.00,5,0.0,70,1,154.739596,41.695857
3,4,Самец,Герефорд,2022-10-08,3959,1785,22.625400,2,0.12,0.00,9,82.0,84,2,37.049199,57.422813
4,5,Самка,Айрширская,2020-10-07,6921,4409,0.000000,3,0.14,0.00,6,0.0,87,4,182.700000,0.000000
5,6,Самка,Голштинская,2019-08-26,7942,7035,35.590611,5,0.01,0.00,7,88.0,86,5,45.181223,0.000000
6,7,Самец,Герефорд,2022-03-04,9472,9004,0.000000,4,0.11,0.00,5,0.0,85,2,184.200000,35.027144


In [20]:
filtered_data = fenotype_data[(fenotype_data['Порода'] == my_cow['Порода'])]

sorted_data = filtered_data.sort_values(by='Compatibility', ascending=False)
sorted_data.head(10)

,ID_особи,Пол,Порода,Дата_Рождения,Родитель_папа,Родитель_мама,Удой л/день,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,Здоровье (1-10),Фертильность (%),Генетическая ценность (баллы),Возраст,Penalty,Compatibility
342,343,Самец,Айрширская,2022-03-06,8135,5846,22.124131,4,0.17,0.86,6,92.0,74,2,64.711737,99.703706
178,179,Самец,Айрширская,2023-08-10,6546,377,25.126601,4,0.07,0.00,9,95.0,72,1,64.146798,99.421572
7837,7838,Самец,Айрширская,2022-10-19,4210,6629,37.189950,3,0.13,0.00,7,93.0,74,2,64.079901,99.317887
8651,8652,Самец,Айрширская,2019-09-03,6876,6564,23.073647,5,0.16,0.00,8,90.0,70,5,63.752705,98.810765
8902,8903,Самец,Айрширская,2018-07-27,3832,5903,21.091644,4,0.02,0.00,5,90.0,74,6,65.416713,98.629230
3566,3567,Самец,Айрширская,2021-11-14,8894,4489,20.340583,4,0.06,0.00,6,95.0,91,3,65.518833,98.475502
2958,2959,Самец,Айрширская,2018-10-01,4201,4345,25.467679,5,0.03,1.37,5,95.0,76,6,63.534643,98.472788
3024,3025,Самец,Айрширская,2018-09-18,7018,7345,37.042830,3,0.01,1.44,7,92.0,73,6,63.525659,98.458864
586,587,Самец,Айрширская,2022-01-17,5502,954,39.529599,4,0.15,0.00,9,92.0,76,2,62.759197,97.270919
5320,5321,Самец,Айрширская,2018-04-14,1535,7703,26.031256,4,0.02,0.00,10,95.0,72,6,62.337488,96.617310
